In [2]:
import numpy as np
import pandas as pd
from keras.models import load_model
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras.models import load_model

def evaluate():
    # Input the csv file
    """
    Sample evaluation function
    Don't modify this function
    """
    df = pd.read_csv('sample_input.csv')
     
    actual_close = np.loadtxt('sample_close.txt')
    
    pred_close = predict_func(df)
    
    # Calculation of squared_error
    actual_close = np.array(actual_close)
    pred_close = np.array(pred_close)
    mean_square_error = np.mean(np.square(actual_close-pred_close))


    pred_prev = [df['Close'].iloc[-1]]
    pred_prev.append(pred_close[0])
    pred_curr = pred_close
    
    actual_prev = [df['Close'].iloc[-1]]
    actual_prev.append(actual_close[0])
    actual_curr = actual_close

    # Calculation of directional_accuracy
    pred_dir = np.array(pred_curr)-np.array(pred_prev)
    actual_dir = np.array(actual_curr)-np.array(actual_prev)
    dir_accuracy = np.mean((pred_dir*actual_dir)>0)*100

    print(f'Mean Square Error: {mean_square_error:.6f}\nDirectional Accuracy: {dir_accuracy:.1f}')

def predict_func(data):
    """
    Modify this function to predict closing prices for next 2 samples.
    Take care of null values in the sample_input.csv file which are listed as NAN in the dataframe passed to you 
    Args:
        data (pandas Dataframe): contains the 50 continuous time series values for a stock index

    Returns:
        list (2 values): your prediction for closing price of next 2 samples
"""
    model = load_model('gru_model.h5')
    
    ndata = data.drop(data.columns[0], axis=1)
    
    df_interpolated = ndata.interpolate(method='linear')
    imputer = SimpleImputer(strategy='mean')
    ndf = pd.DataFrame(imputer.fit_transform(df_interpolated), columns=df_interpolated.columns)

    df_close = ndf['Close'].tolist()
    
    scaler = MinMaxScaler(feature_range=(0,1))
    df_close_transform = scaler.fit_transform(np.array(df_close).reshape(-1,1))  
    
    steps = 50
    x_input = df_close_transform[len(df_close_transform)-steps:].reshape(1,-1)
    
    temp_input = list(x_input)
    temp_input = temp_input[0].tolist()    
    
    # demonstrate prediction for next 10 days
    from numpy import array

    lst_output=[]
    i=0

    while(i<2):

        if(len(temp_input)>steps):
    
            x_input=np.array(temp_input[1:])

            x_input=x_input.reshape(1,-1)
            x_input = x_input.reshape((1, steps, 1))

            yhat = model.predict(x_input, verbose=0)

            temp_input.extend(yhat[0].tolist())
            temp_input=temp_input[1:]

            lst_output.extend(yhat.tolist())
            i=i+1
            
        else:
            x_input = x_input.reshape((1, steps,1))
            yhat = model.predict(x_input, verbose=0)

            temp_input.extend(yhat[0].tolist())

            lst_output.extend(yhat.tolist())
            i=i+1

    ans = scaler.inverse_transform(lst_output).tolist()   
    print(ans)
    return ans
    

if __name__== "__main__":
    evaluate()

[[7262.8576068394805], [7247.4412904134215]]
Mean Square Error: 581.541388
Directional Accuracy: 50.0


C:\Users\Anubha\AppData\Local\Temp/ipykernel_1680/2245882339.py:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pred_dir = np.array(pred_curr)-np.array(pred_prev)


In [3]:
# 7232.200195
# 7277.899902